## Importação das bibliotecas

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

## Criação do Dataframe com Spark

In [ ]:
spark = ( 
 SparkSession
 .builder
 .master("local[*]")
 .appName('spark_dataframe_api')
 .getOrCreate()
)

df = (
    spark
    .read
    .option('delimiter', ';')
    .option('header', 'true')
    .option('inferSchema', 'true')
    .option('enconding', 'ISO-8859-1')
    .csv('./data/precos-gasolina-etanol-01.csv')
)

df.printSchema()

your 131072x1 screen size is bogus. expect trouble
25/03/20 20:21:44 WARN Utils: Your hostname, denis resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/03/20 20:21:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/20 20:21:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/03/20 20:21:45 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/03/20 20:21:45 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


root
 |-- Regiao - Sigla: string (nullable = true)
 |-- Estado - Sigla: string (nullable = true)
 |-- Municipio: string (nullable = true)
 |-- Revenda: string (nullable = true)
 |-- CNPJ da Revenda: string (nullable = true)
 |-- Nome da Rua: string (nullable = true)
 |-- Numero Rua: string (nullable = true)
 |-- Complemento: string (nullable = true)
 |-- Bairro: string (nullable = true)
 |-- Cep: string (nullable = true)
 |-- Produto: string (nullable = true)
 |-- Data da Coleta: string (nullable = true)
 |-- Valor de Venda: string (nullable = true)
 |-- Valor de Compra: string (nullable = true)
 |-- Unidade de Medida: string (nullable = true)
 |-- Bandeira: string (nullable = true)



In [4]:
df.show(5, truncate=False)

+--------------+--------------+----------+-------------------------+-------------------+----------------+----------+-----------+---------+---------+------------------+--------------+--------------+---------------+-----------------+-------------+
|Regiao - Sigla|Estado - Sigla|Municipio |Revenda                  |CNPJ da Revenda    |Nome da Rua     |Numero Rua|Complemento|Bairro   |Cep      |Produto           |Data da Coleta|Valor de Venda|Valor de Compra|Unidade de Medida|Bandeira     |
+--------------+--------------+----------+-------------------------+-------------------+----------------+----------+-----------+---------+---------+------------------+--------------+--------------+---------------+-----------------+-------------+
|N             |AC            |RIO BRANCO|AUTO POSTO AMAPA - EIRELI| 00.529.581/0001-53|VIA CHICO MENDES|3570      |NULL       |AREAL    |69906-119|GASOLINA          |03/01/2022    |6,99          |NULL           |R$ / litro       |VIBRA ENERGIA|
|N             |

## Filtrar Colunas

In [5]:
df_filtered = (
    df.select('Estado - Sigla', 'Produto', 'Valor de Compra', 'Valor de Venda', 'Unidade de Medida')
)

df_filtered.show(5)

+--------------+------------------+---------------+--------------+-----------------+
|Estado - Sigla|           Produto|Valor de Compra|Valor de Venda|Unidade de Medida|
+--------------+------------------+---------------+--------------+-----------------+
|            AC|          GASOLINA|           NULL|          6,99|       R$ / litro|
|            AC|            ETANOL|           NULL|          5,99|       R$ / litro|
|            AC|GASOLINA ADITIVADA|           NULL|          7,05|       R$ / litro|
|            AC|          GASOLINA|           NULL|          6,99|       R$ / litro|
|            AC|            ETANOL|           NULL|          6,12|       R$ / litro|
+--------------+------------------+---------------+--------------+-----------------+
only showing top 5 rows



## Removendo Coluna "Valor de Compra"

In [6]:
(
    df_filtered
    .where(
        F.col('Valor de Compra').isNotNull()
    )
    .show()
)

+--------------+-------+---------------+--------------+-----------------+
|Estado - Sigla|Produto|Valor de Compra|Valor de Venda|Unidade de Medida|
+--------------+-------+---------------+--------------+-----------------+
+--------------+-------+---------------+--------------+-----------------+



In [7]:
df_filtered = df_filtered.drop('Valor de Compra')
df_filtered.show(5)

+--------------+------------------+--------------+-----------------+
|Estado - Sigla|           Produto|Valor de Venda|Unidade de Medida|
+--------------+------------------+--------------+-----------------+
|            AC|          GASOLINA|          6,99|       R$ / litro|
|            AC|            ETANOL|          5,99|       R$ / litro|
|            AC|GASOLINA ADITIVADA|          7,05|       R$ / litro|
|            AC|          GASOLINA|          6,99|       R$ / litro|
|            AC|            ETANOL|          6,12|       R$ / litro|
+--------------+------------------+--------------+-----------------+
only showing top 5 rows



## Convertendo coluna

In [9]:
df_filtered = (
    df_filtered
    .withColumn(
        'Valor de Venda',
        F.regexp_replace(F.col('Valor de Venda'), ',', '.').cast('float')
    )
)

df_filtered.show(5)
df_filtered.printSchema()

+--------------+------------------+--------------+-----------------+
|Estado - Sigla|           Produto|Valor de Venda|Unidade de Medida|
+--------------+------------------+--------------+-----------------+
|            AC|          GASOLINA|          6.99|       R$ / litro|
|            AC|            ETANOL|          5.99|       R$ / litro|
|            AC|GASOLINA ADITIVADA|          7.05|       R$ / litro|
|            AC|          GASOLINA|          6.99|       R$ / litro|
|            AC|            ETANOL|          6.12|       R$ / litro|
+--------------+------------------+--------------+-----------------+
only showing top 5 rows

root
 |-- Estado - Sigla: string (nullable = true)
 |-- Produto: string (nullable = true)
 |-- Valor de Venda: float (nullable = true)
 |-- Unidade de Medida: string (nullable = true)



## Analise sobre o "Valor de Venda"

In [10]:
df_price = (
    df_filtered
    .groupBy(
        F.col('Estado - Sigla'),
        F.col('Produto'),
        F.col('Unidade de Medida')
    )
    .agg(
        F.min(F.col("Valor de Venda")).alias('menor_valor'),
        F.max(F.col("Valor de Venda")).alias('maior_valor')
    )
    .withColumn(
        "diferenca",
        F.col('maior_valor') - F.col('menor_valor')
    )
    .orderBy('diferenca', ascending=False)
)

df_price.show(10)

+--------------+------------------+-----------------+-----------+-----------+---------+
|Estado - Sigla|           Produto|Unidade de Medida|menor_valor|maior_valor|diferenca|
+--------------+------------------+-----------------+-----------+-----------+---------+
|            SP|GASOLINA ADITIVADA|       R$ / litro|      5.789|      8.699|2.9100003|
|            RS|            ETANOL|       R$ / litro|      5.199|      7.797|    2.598|
|            RJ|GASOLINA ADITIVADA|       R$ / litro|      5.799|      8.209|2.4099998|
|            MG|            ETANOL|       R$ / litro|       4.59|      6.999|    2.409|
|            PR|            ETANOL|       R$ / litro|      4.669|       6.99|2.3209996|
|            SP|            ETANOL|       R$ / litro|      4.099|      6.299|2.1999998|
|            RJ|            ETANOL|       R$ / litro|       5.23|      7.359|2.1290002|
|            SP|          GASOLINA|       R$ / litro|      5.489|      7.599|2.1100001|
|            MT|            ETAN